In [1]:
#сделали поддержку большого датасета
#
#здесь мы выгружаем GPT-2 и пытаемся дообучать на кроссдоменном датасете
import torch
from transformers import GPT2Tokenizer, PreTrainedTokenizer
import codecs
import numpy as np
import os, pickle, random
import psutil
import warnings
from typing import Dict, List, Optional
from torch.utils.data import DataLoader
from collections import deque

from trl import AutoModelForCausalLMWithValueHead
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import Trainer, TrainingArguments

import result_gpt_res

In [2]:
torch.cuda.get_device_name(0)

'A100 80GB PCIe'

In [3]:
#with codecs.open('data/all_txt_cut_r.txt', 'r', 'utf8') as f:
#    lines=f.readlines(f)
#print(len(lines))
#del lines

In [4]:
#!pip install transformers
#!pip install trl
#!pip install accelerate -U
#!pip install transformers[torch]

In [5]:
#path_cash = 'ints_cash.bin'
#with open(path_cash, encoding="utf-8") as f:
#    lines = f.readlines()
#lines[0]

In [6]:
import time
#time.sleep(60*5)

In [7]:
#trl - это для RL
#transformers - это для предобучения
#вначале предобучить, затем сконвертировать!
#model_name = "gpt2-medium"
#model_name = "gpt2_finetuned_2"
model_name = "gpt2_finetuned_3"


In [8]:
# 1. load a pretrained model

#model = AutoModelForCausalLM.from_pretrained(model_name)
#model = AutoModelForCausalLM.from_config(model.config)
#model = AutoModelForCausalLMWithValueHead.from_pretrained(model_name)

#model_ref = AutoModelForCausalLMWithValueHead.from_pretrained(model_name)

#tokenizer = GPT2Tokenizer.from_pretrained(model_name)
#tokenizer.pad_token = tokenizer.eos_token
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [9]:
#специальные токены { vertical-output: true }
T_OUT = tokenizer.encode('<OUT>')[0]
T_END = tokenizer.encode('<END>')[0]
T_PAD = tokenizer.encode('|PAD|')[0]
T_124 = tokenizer.encode('<124>')[0]

In [10]:
reward_to_substr = {}
rmax = 25
rmin = -25
step = 0.1
for r in np.arange(rmin, rmax, step):
    r = np.round(r, 2)
    str_r = f'<r{r}>'
    reward_to_substr[r] = str_r
substr_to_reward = {v: k for k, v in reward_to_substr.items()}
print('substr_to_reward', substr_to_reward)
print('reward_to_substr', reward_to_substr)

substr_to_reward {'<r-25.0>': -25.0, '<r-24.9>': -24.9, '<r-24.8>': -24.8, '<r-24.7>': -24.7, '<r-24.6>': -24.6, '<r-24.5>': -24.5, '<r-24.4>': -24.4, '<r-24.3>': -24.3, '<r-24.2>': -24.2, '<r-24.1>': -24.1, '<r-24.0>': -24.0, '<r-23.9>': -23.9, '<r-23.8>': -23.8, '<r-23.7>': -23.7, '<r-23.6>': -23.6, '<r-23.5>': -23.5, '<r-23.4>': -23.4, '<r-23.3>': -23.3, '<r-23.2>': -23.2, '<r-23.1>': -23.1, '<r-23.0>': -23.0, '<r-22.9>': -22.9, '<r-22.8>': -22.8, '<r-22.7>': -22.7, '<r-22.6>': -22.6, '<r-22.5>': -22.5, '<r-22.4>': -22.4, '<r-22.3>': -22.3, '<r-22.2>': -22.2, '<r-22.1>': -22.1, '<r-22.0>': -22.0, '<r-21.9>': -21.9, '<r-21.8>': -21.8, '<r-21.7>': -21.7, '<r-21.6>': -21.6, '<r-21.5>': -21.5, '<r-21.4>': -21.4, '<r-21.3>': -21.3, '<r-21.2>': -21.2, '<r-21.1>': -21.1, '<r-21.0>': -21.0, '<r-20.9>': -20.9, '<r-20.8>': -20.8, '<r-20.7>': -20.7, '<r-20.6>': -20.6, '<r-20.5>': -20.5, '<r-20.4>': -20.4, '<r-20.3>': -20.3, '<r-20.2>': -20.2, '<r-20.1>': -20.1, '<r-20.0>': -20.0, '<r-19.9>': -

In [11]:
#Учим эту модель. У Nehc есть пример, как это делать
#Модификация токенайзера 
#добавляем несколько токенов нашей разметки

#сюда надо фигануть весь словарь картинок
video_tokens_cnt = 1024
video_tokens = []
for i in range(video_tokens_cnt):
    video_tokens.append(f'<{i}>')
    
special_tokens_dict = {'additional_special_tokens': video_tokens + ['<IN>','<OUT>','<END>','|PAD|'] + list(reward_to_substr.values())}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
#print(tokenizer.all_special_tokens)
tokenizer.pad_token = '|PAD|'
#model.resize_token_embeddings(len(tokenizer))

In [12]:
r_max_token = tokenizer.encode(list(reward_to_substr.values())[-1])[0]
r_min_token = tokenizer.encode(list(reward_to_substr.values())[0])[0]
out_token = tokenizer.encode('<OUT>')[0]

In [13]:
#r_mapping {1231:0.1, 1547:1.2} и так далее.
#маппинг с номеров токенов на реворды
r_mapping = {}
for substr in list(reward_to_substr.values()):
    r_token = tokenizer.encode(substr)[0]
    r_mapping[r_token] = substr_to_reward[substr]
print(r_mapping)

{51685: -25.0, 51686: -24.9, 51687: -24.8, 51688: -24.7, 51689: -24.6, 51690: -24.5, 51691: -24.4, 51692: -24.3, 51693: -24.2, 51694: -24.1, 51695: -24.0, 51696: -23.9, 51697: -23.8, 51698: -23.7, 51699: -23.6, 51700: -23.5, 51701: -23.4, 51702: -23.3, 51703: -23.2, 51704: -23.1, 51705: -23.0, 51706: -22.9, 51707: -22.8, 51708: -22.7, 51709: -22.6, 51710: -22.5, 51711: -22.4, 51712: -22.3, 51713: -22.2, 51714: -22.1, 51715: -22.0, 51716: -21.9, 51717: -21.8, 51718: -21.7, 51719: -21.6, 51720: -21.5, 51721: -21.4, 51722: -21.3, 51723: -21.2, 51724: -21.1, 51725: -21.0, 51726: -20.9, 51727: -20.8, 51728: -20.7, 51729: -20.6, 51730: -20.5, 51731: -20.4, 51732: -20.3, 51733: -20.2, 51734: -20.1, 51285: -20.0, 51286: -19.9, 51287: -19.8, 51288: -19.7, 51289: -19.6, 51290: -19.5, 51291: -19.4, 51292: -19.3, 51293: -19.2, 51294: -19.1, 51295: -19.0, 51296: -18.9, 51297: -18.8, 51298: -18.7, 51299: -18.6, 51300: -18.5, 51301: -18.4, 51302: -18.3, 51303: -18.2, 51304: -18.1, 51305: -18.0, 51306

In [14]:
print('mem', result_gpt_res.getmem())
model = result_gpt_res.AutoModelForCausalLMWithValueHeadCL.from_pretrained(model_name)
#r_mapping {1231:0.1, 1547:1.2} и так далее.
#маппинг с номеров токенов на реворды
model.r_mapping = r_mapping
#токен <OUT>, с него начинается прогноз
model.out_token = out_token

model._modules['pretrained_model'].resize_token_embeddings(len(tokenizer))

mem 5.435943603515625e-05


Some weights of the model checkpoint at gpt2_finetuned_3 were not used when initializing GPT2LMHeadModel: ['v_head.2.bias', 'v_head.4.weight', 'v_head.4.bias', 'v_head.0.bias', 'v_head.2.weight', 'v_head.0.weight']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 51785. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer

Embedding(51785, 1024)

In [15]:
ones = torch.ones([122, 122, 1024], device=model.v_head[0].weight.device)
print('ones', model.v_head(ones)[0,0,0:3])

ones tensor([0.2236], grad_fn=<SliceBackward0>)


In [ ]:
from transformers import TextDataset,DataCollatorForLanguageModeling#,LineByLineTextDataset
print('mem', result_gpt_res.getmem())
batch_size = 124#110#128#135 is moo much
batch_size = 61#block_size=256
batch_size = 62
#2.28 GB (2.3 - перебор), всё работает

#train_dataset = TextIterableDataset(
#          tokenizer=tokenizer,
#          args={'overwrite_cache':True, 'model_type':'gpt2_medium'},
#          file_path='data/parts/all_txt.txt',
#          block_size=batch_size)
train_dataset = result_gpt_res.LineByLineTextDataset(
          tokenizer=tokenizer,
          #file_path='data/parts/all_txt.txt',
          #file_path='data/all_txt_low.txt',
          file_path='data/all_txt_cut_r.txt',
          #file_path='data/all_txt_cut_r_low.txt',
          block_size=256)
print(1)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

mem 0.00023651123046875


In [ ]:
#обучение
print('mem', result_gpt_res.getmem())
# если не лезет в память, можно уменьшать **per_device_train_batch_size**
model.tokenizer = tokenizer
training_args = TrainingArguments(
    output_dir="./models/gpt_bot", #The output directory
    #evaluation_strategy="epoch",
    #fp16=True,######
    #fp16_full_eval=True,
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=800, # number of training epochs
    per_device_train_batch_size= batch_size,#70, # batch size for training
    gradient_accumulation_steps=80,#70,#75,#70,#12,
    #per_device_eval_batch_size=50,  # batch size for evaluation
    eval_steps = 3, # Number of update steps between two evaluations.
    save_steps=1000, # after # steps model is saved 
    warmup_steps=200,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )

trainer = Trainer(#result_gpt_res.TrainerUnsampled(
    model=model,
    args=training_args,
    data_collator=data_collator,
    #train_dataset=tokenized_dataset['train'],
    train_dataset=train_dataset,
    #eval_dataset=test_dataset,
    #compute_metrics=compute_metrics,
)

torch.cuda.empty_cache()

In [ ]:
#mem 0.0034456253051757812

In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#0f814d9f404b726f1beb594c4a9edab08db81a07 #14:47

In [ ]:
for i in range(10000):
    #try:
        trainer.train()
    #except Exception as e:
    #    print(e)
        # save it locally
    #    model.save_pretrained("gpt2_finetuned_3")
    #    tokenizer.save_pretrained('gpt2_finetuned_3')

In [ ]:
ones = torch.ones([122, 122, 1024], device=model.v_head[0].weight.device)
print('ones', model.v_head(ones)[0,0,0:3])
#0.2236

In [ ]:
#можно засейвить модель и потом применять в RL
if 1:
    # save it locally
    model.save_pretrained("gpt2_finetuned_3")
    tokenizer.save_pretrained('gpt2_finetuned_3')

    # load the model from the Hub
    #from transformers import AutoModelForCausalLM

    #model = AutoModelForCausalLM.from_pretrained("my-fine-tuned-model-ppo")
1

In [ ]:
def answer(text):
    #text = '\n' + text
    inpt = tokenizer.encode(text, return_tensors="pt")
    inpt= inpt.cuda()
    print(text)

    #out = model.generate(inpt,  max_length=len(inpt[0])+300, do_sample=True, top_k=5, top_p=0.95, temperature=1, eos_token_id=T_END, pad_token_id=T_PAD)
    out = model.generate(inpt,  max_length=len(inpt[0])+300, do_sample=False, top_k=1, top_p=0.95, temperature=0.0001, eos_token_id=T_END, pad_token_id=T_PAD)

    out_tokens = torch.where(out[0]==T_OUT)
    #last_repl = out[0][out_tokens[0][-1]+1:-1]
    #print(out)
    last_repl = out[0][out_tokens[0][-1]+1:-1]
    repl = tokenizer.decode(last_repl)

    print(repl)

In [ ]:
#a11
text = '<916><61><922><922><922><922><922><922><55><906><1022><830><376><830><830><830><830><61><771><402><438><751><830><925><925><830><61><580><519><920><462><830><925><925><925><61><120><519><255><327><61><925><328><55><483><657><255><49><328><830><925><147><120><287><328><830><328><61><830><706><957><29><222><925><55><55><55><328><851><590><596><164><376><61><328><328><328><801><925><422><255> description:'
text = f"<IN>{text}<OUT>"
answer(text)
text = '<916><61><922><922><922><922><922><922><55><906><1022><830><376><830><830><830><830><61><771><402><438><751><830><925><925><830><61><580><519><920><462><830><925><925><925><61><120><519><255><327><61><925><328><55><483><657><255><49><328><830><925><147><120><287><328><830><328><61><830><706><957><29><222><925><55><55><55><328><851><590><596><164><376><61><328><328><328><801><925><422><255> save hp plan:'
text = f"<IN>{text}<OUT>"
answer(text)

In [ ]:
text = '<705><993><941><325><418><751><937><666><681><110><418><623><72><577><261><358><313><325><974><701><884><414><564><414><63><450><624><176><806><894><564><966><626><957><638><749><66><893><924><634><595><143><120><433><202><556><328><910><57><63><692><189><294><414><751><812><49><909><125><797><261><681><503><552><761><382><657><92><343><671><473><761><505><894><857><788><499><937><334><459><746> description:'
text = f"<IN>{text}<OUT>"
answer(text)


In [ ]:
text = '<32><794><685><585><276><610><578><414><769><129><450><681><966><813><250><256><862><749><438><639><551><54><794><887><500><681><520><996><957><202><848><110><985><164><54><807><520><382><508><473><725><358><110><325><597><54><937><49><473><684><156><481><417><498><718><746><404><455><745><725><363><666><325><560><176><848><438><140><172><848><473><714><872><737><120><477><136><813><893><120><746> description:'
text = f"<IN>{text}<OUT>"
answer(text)



In [ ]:
Итак, основная идея в том, что мы генерим description и пишем его после картинки, но до плана. Фишка в том, что зная description, можно уже принимать нихреновые решения. Но у нас description не отличается подробностью (1), и (2) в нём могут быть дублирующиеся объекты. Чем подробнее description, тем он дольше генерится, и тем меньше их влезет в память. Но лучше по нему можно принимать решения, с большей детальностью.
Как его генерить эффективно?
1) Группировать объекты. Перечислить, что у нас ahead, left-ahead, at left, top-left ahead, far left-ahead и так далее. Это можно сделать через dict из counter (чтобы не писать imp at left, imp at left, а написать сразу количество)
2) Сделать токенами длинные, но частотные слова. Right, left, ahead, turn, какие там ещё команды будут.
3) В п1 мы разделили экран на 9 зон по x и y (или на 18 зон, если ещё и по глубине). Можно в эти зоны добавить объекты "стена", "дырка", "препятствие" - их можно описать через геометрию.

In [ ]:
1) +Реворды записывать для каждого кадра, в стиле sarsa (проверь! И убедись, что разные задачи записываются)
2) -В summarize учитывать goal (потому что иначе хз, как предсказывать реворды. И убедись, что это работает в инференсе)
3) -Инференс на CPU+

In [ ]:
Разберись, наконец, с GPU/CPU!+
Пойми, где лаги (в CPU режиме), убедись, что там точно нет перехода на GPU. Докажи это, блин.+
Если проблема не решится - перейди на GPU. Выясни, опять же, где лаги.+